<a href="https://colab.research.google.com/github/ArpitRawat07/Web-Scraping-Rera/blob/main/web_scraping_rera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install playwright
!playwright install chromium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 19.1 MB/s eta 0:00:00
173.7 MiB [] 0% 132.2s173.7 MiB [] 0% 50.3s173.7 MiB [] 0% 23.6s173.7 MiB [] 0% 14.4s173.7 MiB [] 0% 7.6s173.7 MiB [] 1% 5.1s173.7 MiB [] 2% 4.0s173.7 MiB [] 3% 3.3s173.7 MiB [] 4% 2.9s173.7 MiB [] 5% 2.7s173.7 MiB [] 5% 2.9s173.7 MiB [] 6% 2.7s173.7 MiB [] 7% 2.6s173.7 MiB [] 8% 2.4s173.7 MiB [] 8% 2.3s173.7 MiB [] 9% 2.3s173.7 MiB [] 10% 2.3s173.7 MiB [] 11% 2.1s173.7 MiB [] 12% 2.0s173.7 MiB [] 13% 2.1s173.7 MiB [] 14% 2.0s173.7 MiB [] 15% 1.9s173.7 MiB [] 16% 1.8s173.7 MiB [] 17% 1.7s173.7 MiB [] 19% 1.7s173.7 MiB [] 20% 1.6s173.7 MiB [] 21% 1.6s173.7 MiB [] 22% 1.6s173.7 MiB [] 23% 1.5s173.7 MiB [] 24% 1.5s173.7 MiB [] 25% 1.4s173.7 MiB [] 26% 1.5s173.7 MiB [] 27% 1.5s173.7 MiB [] 28% 1.4s173.7 MiB [] 30% 1.3s173.7 MiB [] 31% 1.3s173.7 MiB [] 32% 1.2s173.7 MiB [] 34% 1.2s173.7 MiB [] 35% 1.1s173.7 MiB [] 37% 1.1s173.7 MiB [] 38% 1.1s173.7 MiB [] 40% 1.0s173.7 MiB [] 41% 1.0s173.7 MiB [] 42% 1.0s173.7 Mi

In [ ]:
import csv
import re
import pytesseract
from PIL import Image as PILImage
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import os

# Find all data entries in maharera

In [ ]:
PROGRESS_FILE = "progress.txt"

async def scrape_projects_to_csv(base_url, start_id=1, csv_filename="projects.csv"):
    # --- Resume from last progress if exists ---
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as f:
            saved_id = f.read().strip()
            if saved_id.isdigit():
                start_id = int(saved_id)
                print(f"Resuming from saved Project ID: {start_id}")

    # --- Ensure CSV has header ---
    fieldnames = [
        "Registration Number", "Date of Registration", "Project Status", "Project Name",
        "Project Type", "Project Location",
        "Proposed Completion Date (Original)", "Proposed Completion Date (Revised)",
        "Planning Authority", "Full Name of Planning Authority", "Land Area", "Project Address",
        "District", "Taluka", "Village", "Pin Code",
        "Longitude", "Latitude", "Promoter Name", "Promoter Type",
        "Promoter Address", "Bank IFSC Code", "Permissible Built-up Area"
    ]
    if not os.path.exists(csv_filename):
        with open(csv_filename, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=fieldnames).writeheader()

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        current_id = start_id
        while True:
            url = f"{base_url.rstrip('/')}/{current_id}"
            print(f"\nFetching: {url}")
            await page.goto(url, timeout=60000)

            # --- Stop if no project found ---
            if "Project details not found" in await page.content():
                print(f"Stopped at {current_id}: No more project details found.")
                break

            # --- Solve captcha with up to 3 retries ---
            success = False
            for attempt in range(3):
                try:
                    await page.wait_for_selector("canvas#captcahCanvas", timeout=15000)
                except:
                    print("Captcha canvas not found, skipping this ID.")
                    break

                # Screenshot canvas
                canvas = await page.query_selector("canvas#captcahCanvas")
                await canvas.screenshot(path="canvas.png")

                # OCR
                image = PILImage.open("canvas.png")
                raw_text = pytesseract.image_to_string(image, config='--psm 6')
                alnum_text = ''.join(re.findall(r'[A-Za-z0-9]', raw_text))
                print(f"Captcha OCR (try {attempt+1}):", alnum_text)

                # Fill captcha and submit
                await page.locator('input[name="captcha"]').fill(alnum_text)
                await page.get_by_role("button", name="Submit").click()
                await page.wait_for_timeout(3000)

                # If error popup appears
                if await page.get_by_role("button", name="Ok").is_visible():
                    print("Invalid captcha, retrying...")
                    await page.get_by_role("button", name="Ok").click()
                    await page.wait_for_timeout(1000)
                    continue  # Try again

                success = True
                break

            if not success:
                print(f"Skipping project {current_id} due to repeated captcha failures.")
                current_id += 1
                continue

            # --- Extract project details ---
            html = await page.content()
            soup = BeautifulSoup(html, 'html.parser')

            def get_text_after(label):
                el = soup.find(text=re.compile(label, re.I))
                if el and el.find_next():
                    return el.find_next().get_text(strip=True)
                return None

            def get_value_after_label(label_text):
                label_div = soup.find("div", string=re.compile(label_text, re.I))
                if label_div:
                    value_div = label_div.find_next_sibling("div")
                    if value_div:
                        return value_div.get_text(strip=True)
                return None

            project_details = {
                "Registration Number": get_text_after("Registration Number"),
                "Date of Registration": get_text_after("Date of Registration"),
                "Project Status": get_text_after("Project Status"),
                "Project Name": get_text_after("Project Name"),
                "Project Type": get_text_after("Project Type"),
                "Project Location": get_text_after("Project Location"),
                "Proposed Completion Date (Original)": get_value_after_label(r"Proposed Completion Date \(Original\)"),
                "Proposed Completion Date (Revised)": get_value_after_label(r"Proposed Completion Date \(Revised\)"),
                "Planning Authority": get_text_after("Planning Authority"),
                "Full Name of Planning Authority": get_text_after("Full Name of the Planning Authority"),
                "Land Area": get_text_after("Total Land Area of Approved Layout"),
                "Project Address": get_text_after("Address"),
                "District": get_text_after("District"),
                "Taluka": get_text_after("Taluka"),
                "Village": get_text_after("Village"),
                "Pin Code": get_text_after("Pin Code"),
                "Longitude": get_text_after("Longitude"),
                "Latitude": get_text_after("Latitude"),
                "Promoter Name": get_text_after("Name of Partnership"),
                "Promoter Type": get_text_after("Promoter Type"),
                "Promoter Address": get_text_after("Promoter Official Communication Address"),
                "Bank IFSC Code": get_text_after("IFSC Code"),
            }

            # --- Append to CSV ---
            with open(csv_filename, "a", newline="", encoding="utf-8") as f:
                csv.DictWriter(f, fieldnames=fieldnames).writerow(project_details)

            # --- Save progress ---
            with open(PROGRESS_FILE, "w") as f:
                f.write(str(current_id + 1))

            print(f"✓ Saved Project ID {current_id}")
            current_id += 1

        await browser.close()

    print("Scraping finished.")

In [ ]:
await scrape_projects_to_csv(
    base_url="https://maharerait.maharashtra.gov.in/public/project/view",
    start_id=1,
    csv_filename="maharera_projects.csv"
)


Resuming from saved Project ID: 11

Fetching: https://maharerait.maharashtra.gov.in/public/project/view/11
Captcha OCR (try 1): 4BFLI2
Invalid captcha, retrying...
Captcha canvas not found, skipping this ID.
Skipping project 11 due to repeated captcha failures.

Fetching: https://maharerait.maharashtra.gov.in/public/project/view/12
Captcha OCR (try 1): CLOF7M
Invalid captcha, retrying...
Captcha OCR (try 2): HFA24G


/tmp/ipython-input-3615969914.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved Project ID 12

Fetching: https://maharerait.maharashtra.gov.in/public/project/view/13
Captcha OCR (try 1): BBEG6G0K
Invalid captcha, retrying...
Captcha OCR (try 2): BDOMLH
Invalid captcha, retrying...
Captcha OCR (try 3): 3M11CB


/tmp/ipython-input-3615969914.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved Project ID 13

Fetching: https://maharerait.maharashtra.gov.in/public/project/view/14
Captcha OCR (try 1): cikDsl
Invalid captcha, retrying...
Captcha OCR (try 2): L63102


...

/tmp/ipython-input-3615969914.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved Project ID 394

Fetching: https://maharerait.maharashtra.gov.in/public/project/view/395
Captcha OCR (try 1): H1IDE20
Invalid captcha, retrying...
Captcha OCR (try 2): 4DFNIC


/tmp/ipython-input-3615969914.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved Project ID 395

Fetching: https://maharerait.maharashtra.gov.in/public/project/view/396
Captcha OCR (try 1): S5EDGBE
Invalid captcha, retrying...
Captcha OCR (try 2): 53FO0FM
Invalid captcha, retrying...
Captcha OCR (try 3): I6LNE9
Invalid captcha, retrying...


TimeoutError: Locator.click: Timeout 30000ms exceeded.
Call log:
  - waiting for get_by_role("button", name="Ok")
    - locator resolved to <a role="button" id="download1" value="download" class="download" _ngcontent-ndp-c22="" title="Click to Download"> Gokuldham-Agreement.pdf </a>
  - attempting click action
    2 × waiting for element to be visible, enabled and stable
      - element is visible, enabled and stable
      - scrolling into view if needed
      - done scrolling
      - <div class="abeyance-msg" _ngcontent-ndp-c22="">…</div> from <div tabindex="-1" id="abeyanceMsg" _ngcontent-ndp-c22="" class="modal fade show">…</div> subtree intercepts pointer events
    - retrying click action
    - waiting 20ms
    2 × waiting for element to be visible, enabled and stable
      - element is not stable
    - retrying click action
      - waiting 100ms
    - waiting for element to be visible, enabled and stable
    - element is not stable
  14 × retrying click action
       - waiting 50

# Data entries for Raheja Group


In [ ]:
from playwright.async_api import async_playwright

async def get_all_raheja_project_links():
    all_links = set()

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # headless=True
        page = await browser.new_page()

        await page.goto("https://maharera.maharashtra.gov.in/projects-search-result", timeout=60000)

        # Type "raheja" in Project Name field
        await page.fill('input[name="project_name"]', "raheja")
        await page.get_by_role("button", name="Search").click()
        await page.wait_for_load_state("networkidle")

        page_num = 1
        while True:
            print(f"Scraping page {page_num}...")
            await page.wait_for_selector('a.click-projectmodal.viewLink.targetBlankLink', timeout=30000)

            # Collect all links on this page
            links = await page.eval_on_selector_all(
                'a.click-projectmodal.viewLink.targetBlankLink',
                'elements => elements.map(el => el.href)'
            )

            # Filter only the non-isOriginal links
            for link in links:
                if "?isOriginal=true" not in link:
                    all_links.add(link)

            # Check for the "Next" button
            next_button = page.locator('a.next')
            if await next_button.count() > 0 and await next_button.is_enabled():
                await next_button.click()
                await page.wait_for_load_state("networkidle")
                page_num += 1
            else:
                break

        await browser.close()

    return list(all_links)


In [ ]:
links = await get_all_raheja_project_links()
print(f"Total projects found: {len(links)}")
for l in links:
    print(l)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Total projects found: 48
https://maharerait.maharashtra.gov.in/public/project/view/56402
https://maharerait.maharashtra.gov.in/public/project/view/40873
https://maharerait.maharashtra.gov.in/public/project/view/55889
https://maharerait.maharashtra.gov.in/public/project/view/56256
https://maharerait.maharashtra.gov.in/public/project/view/41191
https://maharerait.maharashtra.gov.in/public/project/view/54949
https://maharerait.maharashtra.gov.in/public/project/view/55411
https://maharerait.maharashtra.gov.in/public/project/view/31227
https://maharerait.maharashtra.gov.in/public/project/view/39202
https://maharerait.maharashtra.gov.in/public/project/view/41817
https://maharerait.maharashtra.gov.in/public/project/view/45385
https://maharerait.maharashtra.gov.in/public/project/view/45797
https://maharerait.maharashtra.gov.in/public/project/view/30577
https://maharerait.maharashtra.gov.in/public/pro

In [ ]:
import os
import csv
import re
from bs4 import BeautifulSoup
from PIL import Image as PILImage
import pytesseract
from playwright.async_api import async_playwright

RAHEJA_PROGRESS_FILE = "raheja_progress.txt"

async def scrape_raheja_projects(links, csv_filename="raheja.csv"):
    # --- Resume from last progress if exists ---
    start_index = 0
    if os.path.exists(RAHEJA_PROGRESS_FILE):
        with open(RAHEJA_PROGRESS_FILE, "r") as f:
            saved_index = f.read().strip()
            if saved_index.isdigit():
                start_index = int(saved_index)
                print(f"Resuming from saved link index: {start_index}")

    # --- Ensure CSV has header ---
    fieldnames = [
        "Registration Number", "Date of Registration", "Project Status", "Project Name",
        "Project Type", "Project Location",
        "Proposed Completion Date (Original)", "Proposed Completion Date (Revised)",
        "Planning Authority", "Full Name of Planning Authority", "Land Area", "Project Address",
        "District", "Taluka", "Village", "Pin Code",
        "Longitude", "Latitude", "Promoter Name", "Promoter Type",
        "Promoter Address", "Bank IFSC Code", "Permissible Built-up Area"
    ]
    if not os.path.exists(csv_filename):
        with open(csv_filename, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=fieldnames).writeheader()

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for idx in range(start_index, len(links)):
            url = links[idx]
            print(f"\nFetching ({idx+1}/{len(links)}): {url}")
            await page.goto(url, timeout=60000)

            # --- Solve captcha if exists (max 3 retries) ---
            success = False
            for attempt in range(10):
                try:
                    await page.wait_for_selector("canvas#captcahCanvas", timeout=5000)
                except:
                    # No captcha found
                    success = True
                    break

                canvas = await page.query_selector("canvas#captcahCanvas")
                await canvas.screenshot(path="canvas.png")
                image = PILImage.open("canvas.png")
                raw_text = pytesseract.image_to_string(image, config='--psm 6')
                alnum_text = ''.join(re.findall(r'[A-Za-z0-9]', raw_text))
                print(f"Captcha OCR (try {attempt+1}): {alnum_text}")

                await page.locator('input[name="captcha"]').fill(alnum_text)
                await page.get_by_role("button", name="Submit").click()
                await page.wait_for_timeout(3000)

                if await page.get_by_role("button", name="Ok").is_visible():
                    print("Invalid captcha, retrying...")
                    await page.get_by_role("button", name="Ok").click()
                    await page.wait_for_timeout(1000)
                    continue

                success = True
                break

            if not success:
                print(f"Skipping {url} due to captcha failures.")
                continue

            # --- Extract project details ---
            html = await page.content()
            soup = BeautifulSoup(html, 'html.parser')

            def get_text_after(label):
                el = soup.find(text=re.compile(label, re.I))
                if el and el.find_next():
                    return el.find_next().get_text(strip=True)
                return None

            def get_value_after_label(label_text):
                label_div = soup.find("div", string=re.compile(label_text, re.I))
                if label_div:
                    value_div = label_div.find_next_sibling("div")
                    if value_div:
                        return value_div.get_text(strip=True)
                return None

            def get_permissible_builtup_area(soup):
              label = soup.find("label", string=re.compile(r"Permissible Built-up Area", re.I))
              if label:
                  container = label.find_next("div", class_="col-12 text-font f-w-700")
                  if container:
                      return container.get_text(strip=True)
              return None


            project_details = {
                "Registration Number": get_text_after("Registration Number"),
                "Date of Registration": get_text_after("Date of Registration"),
                "Project Status": get_text_after("Project Status"),
                "Project Name": get_text_after("Project Name"),
                "Project Type": get_text_after("Project Type"),
                "Project Location": get_text_after("Project Location"),
                "Proposed Completion Date (Original)": get_value_after_label(r"Proposed Completion Date \(Original\)"),
                "Proposed Completion Date (Revised)": get_value_after_label(r"Proposed Completion Date \(Revised\)"),
                "Planning Authority": get_text_after("Planning Authority"),
                "Full Name of Planning Authority": get_text_after("Full Name of the Planning Authority"),
                "Land Area": get_text_after("Total Land Area of Approved Layout"),
                "Project Address": get_text_after("Address"),
                "District": get_text_after("District"),
                "Taluka": get_text_after("Taluka"),
                "Village": get_text_after("Village"),
                "Pin Code": get_text_after("Pin Code"),
                "Longitude": get_text_after("Longitude"),
                "Latitude": get_text_after("Latitude"),
                "Promoter Name": get_text_after("Name of Partnership"),
                "Promoter Type": get_text_after("Promoter Type"),
                "Promoter Address": get_text_after("Promoter Official Communication Address"),
                "Bank IFSC Code": get_text_after("IFSC Code"),
                "Permissible Built-up Area": get_permissible_builtup_area(soup),
            }

            # --- Append to CSV ---
            with open(csv_filename, "a", newline="", encoding="utf-8") as f:
                csv.DictWriter(f, fieldnames=fieldnames).writerow(project_details)

            # --- Save progress ---
            with open(RAHEJA_PROGRESS_FILE, "w") as f:
                f.write(str(idx + 1))

            print(f"✓ Saved project ({idx+1}/{len(links)})")

        await browser.close()

    print("Scraping finished.")


In [ ]:
await scrape_raheja_projects(links, csv_filename="raheja.csv")



Fetching (1/48): https://maharerait.maharashtra.gov.in/public/project/view/56402
Captcha OCR (try 1): I1BICJ


/tmp/ipython-input-3046506841.py:83: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved project (1/48)

Fetching (2/48): https://maharerait.maharashtra.gov.in/public/project/view/40873
Captcha OCR (try 1): 62LOKD
Invalid captcha, retrying...
Captcha OCR (try 2): A43562


/tmp/ipython-input-3046506841.py:83: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved project (2/48)

Fetching (3/48): https://maharerait.maharashtra.gov.in/public/project/view/55889
Captcha OCR (try 1): 93CNBI


...

/tmp/ipython-input-3046506841.py:83: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved project (46/48)

Fetching (47/48): https://maharerait.maharashtra.gov.in/public/project/view/24748
Captcha OCR (try 1): DCASKG
Invalid captcha, retrying...
Captcha OCR (try 2): 7FAKO9
Invalid captcha, retrying...
Captcha OCR (try 3): FHHOGN
Invalid captcha, retrying...
Captcha OCR (try 4): GLONME
Invalid captcha, retrying...
Captcha OCR (try 5): JIH80M
Invalid captcha, retrying...
Captcha OCR (try 6): C104B2
Invalid captcha, retrying...
Captcha OCR (try 7): 9B7F97


/tmp/ipython-input-3046506841.py:83: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))


✓ Saved project (47/48)

Fetching (48/48): https://maharerait.maharashtra.gov.in/public/project/view/40880
Captcha OCR (try 1): D811H1
✓ Saved project (48/48)
Scraping finished.


/tmp/ipython-input-3046506841.py:83: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  el = soup.find(text=re.compile(label, re.I))
